In [52]:
import json
import re
import numpy as np

# Q1: What's the average length of a stage, and how does differ between different tours over the years

In [53]:
def get_grand_tour_distances(grand_tour):
    with open(f'data/{grand_tour}.json', 'r') as file:
        tour_json = json.load(file)

    print(list(tour_json[grand_tour].items())[0])

    distances_per_year = [
        {'year': year[0], 'distances': [course['distance'] for course in year[1]]}
        for year in list(tour_json[grand_tour].items())
    ]

    return distances_per_year

# maybe do with map
france_dict = get_grand_tour_distances('tour_de_france')
italia_dict = get_grand_tour_distances('giro_ditalia')
espagna_dict = get_grand_tour_distances('vuelta_a_espagna')

('1906', [{'key': 1, 'start': 'Paris', 'end': 'Lille', 'distance': '275'}, {'key': 2, 'start': 'Douai', 'end': 'Nancy', 'distance': '400'}, {'key': 3, 'start': 'Nancy', 'end': 'Dijon', 'distance': '416'}, {'key': 4, 'start': 'Dijon', 'end': 'Grenoble', 'distance': '311'}, {'key': 5, 'start': 'Grenoble', 'end': 'Nice', 'distance': '345'}, {'key': 6, 'start': 'Nice', 'end': 'Marseille', 'distance': '292'}, {'key': 7, 'start': 'Marseille', 'end': 'Toulouse', 'distance': '480'}, {'key': 8, 'start': 'Toulouse', 'end': 'Bayonne', 'distance': '300'}, {'key': 9, 'start': 'Bayonne', 'end': 'Bordeaux', 'distance': '338'}, {'key': 10, 'start': 'Bordeaux', 'end': 'Nantes', 'distance': '391'}, {'key': 11, 'start': 'Nantes', 'end': 'Brest', 'distance': '321'}, {'key': 12, 'start': 'Brest', 'end': 'Caen', 'distance': '415'}, {'key': 13, 'start': 'Caen', 'end': 'Paris', 'distance': '259'}])
('1909', [{'key': 1, 'start': 'Milan', 'end': 'Bologna', 'distance': '397'}, {'key': 2, 'start': 'Bologna', 'end

In [54]:
def get_mean_average_per_year(tour_dict):
    print(tour_dict[0])
    [
        tour_dict[i].update({'mean': np.mean(year['distances']), 'average': np.average(year['distances'])})
        for i, year in enumerate(tour_dict, start=0)
    ]

get_mean_average_per_year(france_dict)
get_mean_average_per_year(italia_dict)
get_mean_average_per_year(espagna_dict)

# maybe expand with the average of all

{'year': '1906', 'distances': ['275', '400', '416', '311', '345', '292', '480', '300', '338', '391', '321', '415', '259']}


TypeError: the resolved dtypes are not compatible with add.reduce. Resolved (dtype('<U3'), dtype('<U3'), dtype('<U6'))

# Q2: What rider has finished a tour the most with each jersey in the Tour de France

In [ ]:
with open('jerseys.json', 'r') as f:
    data = json.load(f)

# Yellow Jersey
yellow_data = data['yellow'][0]
max_yellow = max(map(lambda x: int(x['jerseys']), yellow_data))
top_yellow = [r['name'] for r in yellow_data if int(r['jerseys']) == max_yellow]

# White Jersey
white_data = data['white'][0]
max_white = max(white_data.values())
top_white = [name for name, count in white_data.items() if count == max_white]

# Polka Dot Jersey
dotted_data = data['dotted'][0]
max_dotted = max(dotted_data.values())
top_dotted = [name for name, count in dotted_data.items() if count == max_dotted]

# Green Jersey
green_data = data['green'][0]
max_green = max(green_data.values())
top_green = [name for name, count in green_data.items() if count == max_green]

# Results
results = {
    'Yellow': (max_yellow, top_yellow),
    'White': (max_white, top_white),
    'Polka Dot': (max_dotted, top_dotted),
    'Green': (max_green, top_green)
}

for jersey, (count, riders) in results.items():
    print(f"\n{jersey} Jersey - {count} wins:")
    print(f"  {', '.join(riders)}")

# Summary
maxes = np.array([max_yellow, max_white, max_dotted, max_green])
print(f"\nHighest record: {np.max(maxes)} wins | Average: {np.mean(maxes):.1f}")

# Q3 Which riders switch teams most often and which are most loyal of the current riders

In [ ]:
import json
from collections import Counter

with open('rider_teams.json', 'r') as f:
    data = json.load(f)

rider_groups = data['rider'][1:]  # Skip "dummy"

rider_stats = []
for group in rider_groups:
    for rider in group:
        teams = rider['teams']
        rider_stats.append({
            'name': rider['name'],
            'unique_teams': len(set(teams)),
            'total_entries': len(teams),
            'teams': teams
        })

# Most switches (most unique teams)
most_switches = sorted(rider_stats, key=lambda x: x['unique_teams'], reverse=True)[:10]

# Most loyal (fewest unique teams, min 5 entries)
experienced = [r for r in rider_stats if r['total_entries'] >= 5]
most_loyal = sorted(experienced, key=lambda x: x['unique_teams'])[:10]

print("\nTOP 10 TEAM SWITCHERS:")
for i, r in enumerate(most_switches, 1):
    ratio = r['total_entries']/r['unique_teams']
    print(f"{i}. {r['name']}: {r['unique_teams']} teams ({r['total_entries']} entries, {ratio:.1f}x/team)")

print("\nTOP 10 MOST LOYAL (min 5 entries):")
for i, r in enumerate(most_loyal, 1):
    ratio = r['total_entries']/r['unique_teams']
    main_team = Counter(r['teams']).most_common(1)[0]
    print(f"{i}. {r['name']}: {r['unique_teams']} teams ({r['total_entries']} entries, {ratio:.1f}x/team)")
    print(f"   Main: {main_team[0]} ({main_team[1]}x)")

# Summary
avg_unique = sum(r['unique_teams'] for r in rider_stats) / len(rider_stats)
avg_entries = sum(r['total_entries'] for r in rider_stats) / len(rider_stats)
print(f"\nAvg: {avg_unique:.1f} teams, {avg_entries:.1f} entries, {avg_entries/avg_unique:.1f}x/team")

# Q4 How has the age of the average tour winner evolved over the years

In [ ]:
import json
import numpy as np
from collections import defaultdict

with open('ages.json', 'r') as f:
    data = json.load(f)

victors = data['victors'][0]

# Extract year and calculate age
ages_by_decade = defaultdict(list)
all_ages = []

for victor in victors:
    year = int(victor['year'])
    
    # Try to get birthyear - some entries might not have it
    if 'birthyear' in victor and victor['birthyear']:
        birthyear = int(victor['birthyear'])
        age = year - birthyear
        
        decade = (year // 10) * 10
        ages_by_decade[decade].append(age)
        all_ages.append((year, age, victor['rider']))

# Calculate statistics by decade
print("AVERAGE AGE OF TOUR WINNERS BY DECADE:\n")
for decade in sorted(ages_by_decade.keys()):
    ages = ages_by_decade[decade]
    avg = np.mean(ages)
    min_age = min(ages)
    max_age = max(ages)
    print(f"{decade}s: {avg:.1f} years (range: {min_age}-{max_age})")

# Recent trends (last 30 years)
recent_30 = [age for year, age, _ in all_ages if year >= 1995]
older_30 = [age for year, age, _ in all_ages if year < 1995 and year >= 1965]

print(f"\nTREND COMPARISON:")
print(f"1965-1994: {np.mean(older_30):.1f} years average")
print(f"1995-2025: {np.mean(recent_30):.1f} years average")
print(f"Change: {np.mean(recent_30) - np.mean(older_30):+.1f} years")

# Youngest and oldest winners
sorted_ages = sorted(all_ages, key=lambda x: x[1])
print(f"\nYOUNGEST WINNERS:")
for year, age, rider in sorted_ages[:5]:
    print(f"{rider.replace('1', '')}: {age} years ({year})")

print(f"\nOLDEST WINNERS:")
for year, age, rider in sorted_ages[-5:]:
    print(f"{rider.replace('1', '')}: {age} years ({year})")

# Overall statistics
ages_only = [age for _, age, _ in all_ages]
print(f"\nOVERALL STATISTICS:")
print(f"Mean age: {np.mean(ages_only):.1f} years")
print(f"Median age: {np.median(ages_only):.1f} years")
print(f"Std deviation: {np.std(ages_only):.1f} years")

# Q5 How has the nationality of the podiums evolved over the years

In [ ]:
import json
from collections import Counter, defaultdict

with open('ages.json', 'r') as f:
    data = json.load(f)

country_data = data['country'][1]  # Skip null at index 0

# Map country codes to full names
country_names = {
    'fr': 'France', 'be': 'Belgium', 'it': 'Italy', 'es': 'Spain', 'nl': 'Netherlands',
    'gb': 'Great Britain', 'us': 'USA', 'lu': 'Luxembourg', 'ch': 'Switzerland',
    'de': 'Germany', 'ie': 'Ireland', 'au': 'Australia', 'dk': 'Denmark',
    'co': 'Colombia', 'si': 'Slovenia', 'ec': 'Ecuador', 'at': 'Austria',
    'pt': 'Portugal', 'kz': 'Kazakhstan', 'lt': 'Lithuania', 'lv': 'Latvia',
    'pl': 'Poland', 'ru': 'Russia', 'se': 'Sweden'
}

# Analyze by decade
decades = defaultdict(lambda: {'countries': [], 'years': []})

for entry in country_data:
    year = int(entry['year'])
    decade = (year // 10) * 10
    
    for country_code in entry['countries']:
        country = country_names.get(country_code, country_code.upper())
        decades[decade]['countries'].append(country)
        decades[decade]['years'].append(year)

# Display results by decade
print("PODIUM NATIONALITIES BY DECADE:\n")
for decade in sorted(decades.keys()):
    counts = Counter(decades[decade]['countries'])
    total_podiums = len(decades[decade]['countries'])
    
    print(f"{decade}s ({len(set(decades[decade]['years']))} Tours):")
    for country, count in counts.most_common(5):
        pct = (count / total_podiums) * 100
        print(f"  {country}: {count} ({pct:.1f}%)")
    print()

# Era comparisons
eras = {
    'Early (1903-1939)': [y for y in range(1903, 1940)],
    'Post-War (1947-1970)': [y for y in range(1947, 1971)],
    'Modern (1971-2000)': [y for y in range(1971, 2001)],
    'Recent (2001-2025)': [y for y in range(2001, 2026)]
}

print("DOMINANT NATIONS BY ERA:\n")
for era_name, years in eras.items():
    era_countries = []
    for entry in country_data:
        if int(entry['year']) in years:
            era_countries.extend([country_names.get(c, c.upper()) for c in entry['countries']])
    
    if era_countries:
        counts = Counter(era_countries)
        total = len(era_countries)
        print(f"{era_name}:")
        for country, count in counts.most_common(3):
            print(f"  {country}: {count}/{total} ({count/total*100:.1f}%)")
        print()

# Diversity analysis
print("NATIONALITY DIVERSITY:\n")
recent_decades = [1990, 2000, 2010, 2020]
for decade in recent_decades:
    decade_countries = []
    for entry in country_data:
        year = int(entry['year'])
        if decade <= year < decade + 10:
            decade_countries.extend([country_names.get(c, c.upper()) for c in entry['countries']])
    
    unique = len(set(decade_countries))
    print(f"{decade}s: {unique} different countries represented")

# Recent dominance (last 10 years)
recent = [entry for entry in country_data if int(entry['year']) >= 2015]
recent_countries = []
for entry in recent:
    recent_countries.extend([country_names.get(c, c.upper()) for c in entry['countries']])

print(f"\nLAST 10 YEARS (2015-2025):")
for country, count in Counter(recent_countries).most_common():
    print(f"{country}: {count} podiums")